<a href="https://colab.research.google.com/github/chandankumar3it/bank-loan-eda-analysis/blob/main/Bank_Loan_Case_Study_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Required Libraries

In [1]:
#Importing all the important libraries like numpy. pandas, matlplolib, and warnings to keep notebook clean

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
import seaborn as sns

In [2]:
# To suppress warnings

import warnings
warnings.filterwarnings("ignore")

In [ ]:
#notebook setting to display all the rowns and columns to have better clearity on the data.

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)

# Dataset - "application_data.csv"

## Reading and Understanding the Dataset

#### Importing the dataset





In [ ]:
# importing application_data.csv

appData_df = pd.read_csv("application_data.csv")

#### Understanding the dataset

In [ ]:
appData_df.head()

In [ ]:
#Checking the rows and columns of the raw dataset

appData_df.shape

In [ ]:
#Checking information of all the columns like data types
appData_df.info("all")